### 1. Phone Book

In [2]:
class Query:
    def __init__(self, query):
        self.type = query[0]
        self.number = int(query[1])
        if self.type == 'add':
            self.name = query[2]

def read_queries():
    n = int(input())
    return [Query(input().split()) for i in range(n)]

def write_responses(result):
    print('\n'.join(result))

def process_queries(queries):
    result = []
    # Keep list of all existing (i.e. not deleted yet) contacts.
    contacts = []
    for cur_query in queries:
        if cur_query.type == 'add':
            # if we already have contact with such number,
            # we should rewrite contact's name
            for contact in contacts:
                if contact.number == cur_query.number:
                    contact.name = cur_query.name
                    break
            else: # otherwise, just add it
                contacts.append(cur_query)
        elif cur_query.type == 'del':
            for j in range(len(contacts)):
                if contacts[j].number == cur_query.number:
                    contacts.pop(j)
                    break
        else:
            response = 'not found'
            for contact in contacts:
                if contact.number == cur_query.number:
                    response = contact.name
                    break
            result.append(response)
    return result

In [18]:
queries = [
    'add 911 police',
    'add 76213 Mom',
    'add 17239 Bob',
    'find 76213',
    'find 910',
    'find 911',
    'del 910',
    'del 911',
    'find 911',
    'find 76213',
    'add 76213 daddy',
    'find 76213',
]
queries = [Query(query_str.split()) for query_str in queries]

def process_queries(queries):
    results = []
    contacts = {}
    for query in queries:
        if query.type == 'add':
            contacts[query.number] = query.name
        elif query.type == 'find':
            if query.number in contacts:
                results.append(contacts.get(query.number))
            else:
                results.append('not found')
        elif query.type == 'del':
            if query.number in contacts:
                del contacts[query.number]
        else:
            raise ValueError('Operation must be `add`, `find`, or `del`')

    return results

process_queries(queries)

['Mom', 'not found', 'police', 'not found', 'Mom', 'daddy']

### 2. Hashing with chains

In [ ]:
class Query:

    def __init__(self, query):
        self.type = query[0]
        if self.type == 'check':
            self.ind = int(query[1])
        else:
            self.s = query[1]

class QueryProcessor:
    _multiplier = 263
    _prime = 1000000007

    def __init__(self, bucket_count):
        self.bucket_count = bucket_count
        # store all strings in one list
        self.elems = []

    def _hash_func(self, s):
        ans = 0
        for c in reversed(s):
            ans = (ans * self._multiplier + ord(c)) % self._prime
        return ans % self.bucket_count

    def write_search_result(self, was_found):
        print('yes' if was_found else 'no')

    def write_chain(self, chain):
        print(' '.join(chain))

    def read_query(self):
        return Query(input().split())

    def process_query(self, query):
        if query.type == "check":
            # use reverse order, because we append strings to the end
            self.write_chain(cur for cur in reversed(self.elems) if self._hash_func(cur) == query.ind)
        else:
            try:
                ind = self.elems.index(query.s)
            except ValueError:
                ind = -1
            if query.type == 'find':
                self.write_search_result(ind != -1)
            elif query.type == 'add':
                if ind == -1:
                    self.elems.append(query.s)
            else:
                if ind != -1:
                    self.elems.pop(ind)

    def process_queries(self):
        n = int(input())
        for i in range(n):
            self.process_query(self.read_query())

In [ ]:
class QueryProcessor:
    _multiplier = 263
    _prime = 1000000007

    def __init__(self, bucket_count):
        self.bucket_count = bucket_count
        # store all strings in one list
        self.chains = {chain_index: deque() for chain_index in range(bucket_count)}
        self.results = []

    def _hash_func(self, s):
        ans = 0
        for c in reversed(s):
            ans = (ans * self._multiplier + ord(c)) % self._prime
        return ans % self.bucket_count

    def write_search_result(self, was_found):
        print('yes' if was_found else 'no')

    def write_chain(self, chain):
        self.results.append(' '.join(chain))

    def read_query(self):
        return Query(input().split())

    def process_query(self, query):
        if query.type == "check":
            self.write_chain(self.chains[query.ind])

        elif query.type == 'add':
            chain_index = self._hash_func(query.s)
            if query.s not in self.chains[chain_index]:
                self.chains[chain_index].appendleft(query.s)

        elif query.type == 'find':
            chain_index = self._hash_func(query.s)
            if query.s not in self.chains[chain_index]:
                self.results.append('no')
            else:
                self.results.append('yes')

        elif query.type == 'del':
            chain_index = self._hash_func(query.s)
            if query.s in self.chains[chain_index]:
                self.chains[chain_index].remove(query.s)
        
        else:
            raise ValueError('Operation must be one of `add`, `check`, `find`, `del`')
            
    def process_queries(self):
        n = int(input())
        for i in range(n):
            self.process_query(self.read_query())
        self.write_responses()
    
    def write_responses(self):
        print('\n'.join(self.results))


queries = [
    'add world',
    'add HellO',
    'check 4',
    'find World',
    'find world',
    'del world',
    'check 4',
    'del HellO',
    'add luck',
    'add GooD',
    'check 2',
    'del good',
]

bucket_count = 5
proc = QueryProcessor(bucket_count)
proc.process_queries()

### 3. Find pattern in text

In [10]:
# python3

def read_input():
    return (input().rstrip(), input().rstrip())

def print_occurrences(output):
    print(' '.join(map(str, output)))

def get_occurrences(pattern, text):
    return [
        i 
        for i in range(len(text) - len(pattern) + 1) 
        if text[i:i + len(pattern)] == pattern
    ]

if __name__ == '__main__':
    print_occurrences(get_occurrences(*read_input()))

0


In [57]:
def PolyHash(text, polynomial=10, prime=1000000007):
    text_list = list(text)[::-1]
    hash_val = 0
    for char in text_list:
        hash_val = ((hash_val * polynomial) + ord(char)) % prime
    return hash_val

def PrecomputeHash(text, pattern, polynomial=10, prime=1000000007):
    pattern_len = len(pattern)
    text_len = len(text)

    last_substring = text[(text_len-pattern_len):text_len]
    last_substring_hash = PolyHash(last_substring, polynomial, prime)
    
    hash_values = [None] * (text_len-pattern_len+1)
    hash_values[-1] = last_substring_hash
    
    x_power_patternlen = 1
    for i in range(pattern_len):
        x_power_patternlen = (x_power_patternlen * polynomial) % prime 

    for i in range(text_len-pattern_len-1, -1, -1):
        hash_values[i] = (polynomial*hash_values[i+1] + ord(text[i]) - (ord(text[i + pattern_len]) *  x_power_patternlen)) % prime
    
    return hash_values

def RabinKarp(pattern, text, polynomial=123, prime=1000000007):
    hash_values = PrecomputeHash(text, pattern, polynomial=polynomial, prime=prime)
    pattern_hash = PolyHash(pattern, polynomial)
    positions = []

    for i in range(len(text) - len(pattern) + 1):
        # print(f"{i=}, {hash_values[i]=}")
        if hash_values[i] == pattern_hash:
            # print('here')
            if text[i:(i+len(pattern))] == pattern:
                positions.append(i)
    return positions

In [56]:
pattern = 'aba'
text = 'abacaba'

# PolyHash(pattern)
print(PrecomputeHash(pattern=pattern, text=text))
RabinKarp(pattern, text)

[10777, 10968, 10787, 10869, 10777]
i=0, hash_values[i]=1479664
here
i=1, hash_values[i]=1509800
i=2, hash_values[i]=1479787
i=3, hash_values[i]=1494672
i=4, hash_values[i]=1479664
here


[0, 4]

### 4. Substring equality